#**Dataset Description**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import normalize

In [2]:
raw_data = pd.read_csv("Data/kyc.csv")
cash_raw = pd.read_csv("Data/cash_trxns.csv")
emt_raw = pd.read_csv("Data/emt_trxns.csv")
wire_raw = pd.read_csv("Data/wire_trxns.csv")

print(raw_data.shape,cash_raw.shape,emt_raw.shape,wire_raw.shape)

(195789, 7) (212532, 4) (506451, 7) (67872, 8)


In [3]:
raw_data.head()

,Name,Gender,Occupation,Age,Tenure,cust_id,label
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0
1,ANTHONY ADAMS,male,Musician,52.0,8.0,CUST69248708,0
2,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,0
3,STEPHEN FIGUEROA,male,Optometrist,35.0,17.0,CUST33995820,0
4,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1


In [4]:
raw_data.info()
cash_raw.info()
emt_raw.info()
wire_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195789 entries, 0 to 195788
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Name        195789 non-null  object 
 1   Gender      195789 non-null  object 
 2   Occupation  195789 non-null  object 
 3   Age         195789 non-null  float64
 4   Tenure      195789 non-null  float64
 5   cust_id     195789 non-null  object 
 6   label       195789 non-null  int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 10.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212532 entries, 0 to 212531
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   cust_id  212532 non-null  object
 1   amount   212532 non-null  int64 
 2   type     212532 non-null  object
 3   trxn_id  212532 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506451 entr

In [5]:
raw_data.describe()

,Age,Tenure,label
count,195789.000000,195789.000000,195789.000000
mean,35.832856,6.528436,0.028163
std,10.714043,5.998545,0.165439
min,18.000000,0.000000,0.000000
25%,28.000000,2.000000,0.000000
50%,35.000000,5.000000,0.000000
75%,42.000000,10.000000,0.000000
max,92.000000,49.000000,1.000000


In [6]:
pos_data = raw_data[raw_data['label'] == 1]
pos_data

,Name,Gender,Occupation,Age,Tenure,cust_id,label
4,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1
28,ROBERT STAFFORD,male,Offshore Trustee,51.0,9.0,CUST95743620,1
32,AARUSH SRIDHAR,female,Pawn Shop Owner,48.0,25.0,CUST37591597,1
41,FANG JIAN JUN,female,Private Banker,35.0,3.0,CUST14940080,1
69,MICHAEL JONES,male,Private Jet Broker,52.0,0.0,CUST40832249,1
...,...,...,...,...,...,...,...
195592,TAN JIAN GUO,female,Maritime or Shipping Agent,50.0,11.0,CUST39737655,1
195631,JOSEPH ESTES,male,Police Officer,25.0,7.0,CUST19561308,1
195692,AMANDA CRUZ,female,Hedge Fund Manager,49.0,10.0,CUST55221806,1
195714,DR.TINA LEVINE,female,Free Trade Zone Operator,53.0,15.0,CUST92442267,1


In [7]:
pos_data.describe()

,Age,Tenure,label
count,5514.000000,5514.000000,5514.0
mean,41.428908,9.859086,1.0
std,10.984981,7.400461,0.0
min,18.000000,0.000000,1.0
25%,34.000000,4.000000,1.0
50%,41.000000,9.000000,1.0
75%,49.000000,15.000000,1.0
max,80.000000,41.000000,1.0


#**Deal with imbalance issue**

In [8]:
total_counts = raw_data['label'].value_counts()
Class_1 = (raw_data['label'].value_counts()[1] / len(raw_data)) * 100
Class_0 = (raw_data['label'].value_counts()[0] / len(raw_data)) * 100

print("Percentage of risky (1):", Class_1)
print("Percentage of not risky (0):", Class_0)

Percentage of risky (1): 2.816297136202749
Percentage of not risky (0): 97.18370286379725


#**Data Preprocessing**

In [ ]:
emt_raw['Type'] = "EMT"
emt_raw['Frequency'] = 1

# Drop columns from emt_raw
emt_raw = emt_raw.drop(['trxn_id', 'emt message'], axis=1)

# Rename column in emt_raw
emt_raw = emt_raw.rename(columns={'emt value': 'Amount'})

# Define function to determine 'Type' for wire_raw
def determine_type(row):
    if row['country sender'] == row['country receiver']:
        return "Wire International"
    else:
        return "Wire Domestic"

# Set 'Type' column for wire_raw using apply
wire_raw['Type'] = wire_raw.apply(determine_type, axis=1)
wire_raw['Frequency'] = 1

# Drop columns from wire_raw
wire_raw = wire_raw.drop(['country sender', 'country receiver', 'trxn_id'], axis=1)

# Rename column in wire_raw
wire_raw = wire_raw.rename(columns={'wire value': 'Amount'})

# Concatenate DataFrames
sum_transcation_df = pd.concat([emt_raw, wire_raw], ignore_index=True)
sum_transcation_df = sum_transcation_df.sort_values('id sender')
sum_transcation_df.to_csv(path_or_buf='Data/sum_transcation.csv', index=False)

total_table = sum_transcation_df.groupby(['id sender', 'id receiver'])[['Amount','Frequency']].sum().reset_index()
total_table.columns = ['id sender', 'id receiver', 'Sum of Amount', 'Frequency']
total_table = total_table.sort_values('Frequency',ascending=False)
print(total_table)

filter_total_table = total_table[total_table['Sum of Amount'] > 8440]
print(filter_total_table.describe())

filter_total_table.head()
filter_total_table.to_csv(path_or_buf='Data/cleaned_transcation.csv', index=False)


In [9]:
# Cash Table
cash_result = cash_raw.groupby(['cust_id', 'type'])['amount'].sum().reset_index()
cash_send = cash_result[cash_result['type'] == 'withdrawal'][['cust_id','amount']].rename(columns={'amount': 'amount_withdraw'})
cash_receive = cash_result[cash_result['type'] == 'deposit'][['cust_id','amount']].rename(columns={'amount': 'amount_deposite'})

# EMT Table - sender
emt_sender = emt_raw.groupby('id sender')['emt value'].sum().reset_index().rename(columns={'id sender': 'cust_id', 'emt value': 'emt_sender'})

# EMT Table - receive
emt_receive = emt_raw.groupby('id receiver')['emt value'].sum().reset_index().rename(columns={'id receiver': 'cust_id', 'emt value': 'emt_receive'})

# Wire Table - send (aboard or not)
wire_send_total_raw = wire_raw.groupby('id sender')['wire value'].sum().reset_index()
wire_send_total = pd.merge(wire_send_total_raw, wire_raw[['id sender', 'country sender', 'country receiver']], on='id sender', how='left')
wire_send_aboard = wire_send_total[wire_send_total['country sender'] == wire_send_total['country receiver']][['id sender', 'wire value']].rename(columns={'id sender': 'cust_id','wire value': 'wire_send_aboard'})
wire_send_domes = wire_send_total[wire_send_total['country sender'] != wire_send_total['country receiver']][['id sender', 'wire value']].rename(columns={'id sender': 'cust_id','wire value': 'wire_send_domestic'})

# Wire Table - receive (aboard or not)
wire_receive_total_raw = wire_raw.groupby('id receiver')['wire value'].sum().reset_index()
wire_receive_total = pd.merge(wire_receive_total_raw, wire_raw[['id receiver', 'country sender', 'country receiver']], on='id receiver', how='left')
wire_receive_aboard = wire_receive_total[wire_receive_total['country sender'] == wire_receive_total['country receiver']][['id receiver', 'wire value']].rename(columns={'id receiver': 'cust_id','wire value': 'wire_receive_aboard'})
wire_receive_domes = wire_receive_total[wire_receive_total['country sender'] != wire_receive_total['country receiver']][['id receiver', 'wire value']].rename(columns={'id receiver': 'cust_id','wire value': 'wire_receive_domestic'})

In [10]:
print(wire_send_aboard)

              cust_id  wire_send_aboard
1        CUST10002761            9330.5
2        CUST10002761            9330.5
3        CUST10002761            9330.5
4        CUST10005774            5046.0
5        CUST10006702            1890.0
...               ...               ...
67863  EXTERNAL999643            1547.5
67865  EXTERNAL999775            2374.0
67866  EXTERNAL999799            1100.0
67869  EXTERNAL999902           98180.0
67870  EXTERNAL999902           98180.0

[46594 rows x 2 columns]


In [11]:
result_df = (
    pd.merge(raw_data, cash_send, on='cust_id', how='outer')
    .merge(cash_receive, on='cust_id', how='outer')
    .merge(emt_sender, on='cust_id', how='outer')
    .merge(emt_receive, on='cust_id', how='outer')
    .merge(wire_send_aboard, on='cust_id', how='outer')
    .merge(wire_send_domes, on='cust_id', how='outer')
    .merge(wire_receive_aboard, on='cust_id', how='outer')
    .merge(wire_receive_domes, on='cust_id', how='outer')
    .fillna(0)
    .drop_duplicates(subset='cust_id')
)

print(result_df.shape)
result_df.head(50)



(300008, 15)


,Name,Gender,Occupation,Age,Tenure,cust_id,label,amount_withdraw,amount_deposite,emt_sender,emt_receive,wire_send_aboard,wire_send_domestic,wire_receive_aboard,wire_receive_domestic
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0.0,0.0,0.0,14412.0,11484.5,3971.5,0.0,0.0,0.0
2,ANTHONY ADAMS,male,Musician,52.0,8.0,CUST69248708,0.0,0.0,1210.0,0.0,84.0,0.0,0.0,0.0,0.0
3,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,0.0,0.0,3835.0,4494.0,6538.5,28263.0,28263.0,22629.5,22629.5
7,STEPHEN FIGUEROA,male,Optometrist,35.0,17.0,CUST33995820,0.0,650.0,0.0,0.0,0.0,0.0,0.0,0.0,27385.0
8,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1.0,19585.0,68115.0,1147.5,18289.0,10429.0,0.0,23895.5,0.0
14,BECKY SUTTON,female,Sommelier,26.0,8.0,CUST41866546,0.0,0.0,0.0,0.0,21.5,0.0,0.0,0.0,0.0
15,JOCELYN BOWMAN,female,Musician,29.0,11.0,CUST14581764,0.0,0.0,4090.0,0.0,16.5,0.0,0.0,0.0,0.0
16,ZHANG FENG,female,Stock Broker,23.0,5.0,CUST89345714,0.0,4335.0,0.0,250.0,2848.0,2641.0,0.0,0.0,0.0
17,BRIAN OLIVER,male,Antiques Dealer,47.0,10.0,CUST93206512,0.0,19535.0,77225.0,2974.5,2618.0,0.0,2660.0,0.0,0.0
18,MICHAEL RUIZ,male,School Teacher,43.0,7.0,CUST25056739,0.0,0.0,0.0,0.0,404.5,0.0,0.0,0.0,0.0


#**Model Selection**

#**Evaluation**

##**Task 2 (Finding IWT networks)**

### threshold determine

In [12]:
columns_to_sum = ['amount_withdraw', 'amount_deposite',
                  'emt_sender', 'emt_receive', 'wire_send_aboard',
                  'wire_send_domestic',	'wire_receive_aboard',
                  'wire_receive_domestic']

# Sum only the specified columns for each row
result_df['total_amount'] = result_df[columns_to_sum].sum(axis=1)

result_df

,Name,Gender,Occupation,Age,Tenure,cust_id,label,amount_withdraw,amount_deposite,emt_sender,emt_receive,wire_send_aboard,wire_send_domestic,wire_receive_aboard,wire_receive_domestic,total_amount
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0.0,0.0,0.0,14412.0,11484.5,3971.5,0.0,0.0,0.0,29868.0
2,ANTHONY ADAMS,male,Musician,52.0,8.0,CUST69248708,0.0,0.0,1210.0,0.0,84.0,0.0,0.0,0.0,0.0,1294.0
3,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,0.0,0.0,3835.0,4494.0,6538.5,28263.0,28263.0,22629.5,22629.5,116652.5
7,STEPHEN FIGUEROA,male,Optometrist,35.0,17.0,CUST33995820,0.0,650.0,0.0,0.0,0.0,0.0,0.0,0.0,27385.0,28035.0
8,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1.0,19585.0,68115.0,1147.5,18289.0,10429.0,0.0,23895.5,0.0,141461.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353741,0,0,0,0.0,0.0,EXTERNAL998377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10480.0,10480.0
353743,0,0,0,0.0,0.0,EXTERNAL998665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9517.0,9517.0
353744,0,0,0,0.0,0.0,EXTERNAL998972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5140.0,5140.0
353745,0,0,0,0.0,0.0,EXTERNAL999599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3282.0,3282.0


In [13]:
result_df['total_amount'].describe()

count    3.000080e+05
mean     8.440188e+03
std      2.438430e+04
min      0.000000e+00
25%      1.590000e+02
50%      8.342500e+02
75%      4.108000e+03
max      1.510836e+06
Name: total_amount, dtype: float64

In [14]:
## delete rows if this person have zero transaction, becasue it is useless
## for us to detect wildlife trafficking network

filtered_df = result_df[result_df['label'] == 1]

filtered_df

,Name,Gender,Occupation,Age,Tenure,cust_id,label,amount_withdraw,amount_deposite,emt_sender,emt_receive,wire_send_aboard,wire_send_domestic,wire_receive_aboard,wire_receive_domestic,total_amount
8,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1.0,19585.0,68115.0,1147.5,18289.0,10429.0,0.0,23895.5,0.0,141461.0
41,ROBERT STAFFORD,male,Offshore Trustee,51.0,9.0,CUST95743620,1.0,0.0,0.0,17193.0,1623.5,10144.5,0.0,7266.5,0.0,36227.5
48,AARUSH SRIDHAR,female,Pawn Shop Owner,48.0,25.0,CUST37591597,1.0,27090.0,162570.0,9.0,1.0,0.0,0.0,0.0,0.0,189670.0
62,FANG JIAN JUN,female,Private Banker,35.0,3.0,CUST14940080,1.0,22045.0,26120.0,8019.0,1141.5,40495.5,40495.5,19047.5,19047.5,176411.5
116,MICHAEL JONES,male,Private Jet Broker,52.0,0.0,CUST40832249,1.0,14515.0,46800.0,20416.5,5927.0,4664.0,0.0,8944.0,0.0,101266.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233206,TAN JIAN GUO,female,Maritime or Shipping Agent,50.0,11.0,CUST39737655,1.0,30040.0,53365.0,9556.5,8094.0,2268.0,0.0,11393.5,11393.5,126110.5
233255,JOSEPH ESTES,male,Police Officer,25.0,7.0,CUST19561308,1.0,0.0,0.0,1052.0,2305.5,226510.0,0.0,44105.0,44105.0,318077.5
233345,AMANDA CRUZ,female,Hedge Fund Manager,49.0,10.0,CUST55221806,1.0,0.0,0.0,11522.5,11033.5,1172.0,0.0,114819.5,114819.5,253367.0
233373,DR.TINA LEVINE,female,Free Trade Zone Operator,53.0,15.0,CUST92442267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
filtered_df['total_amount'].describe()

count      5514.000000
mean      87842.514418
std       75445.639462
min           0.000000
25%       32071.000000
50%       70880.000000
75%      128637.625000
max      988086.500000
Name: total_amount, dtype: float64

In [16]:
## we decide to use the total_amount average of original data as threshold
## we only consider chose who above threshold for task 2
threshold = result_df['total_amount'].mean()
threshold

8440.188491973548

### occupation selection
we filter out low risk occupation

In [34]:
# filter out low risk occupation
over_threshold = result_df[result_df['total_amount'] > threshold]
occupation_counts = over_threshold['Occupation'].value_counts()

# Convert the result to a DataFrame
occupation_counts_df = occupation_counts.reset_index()
occupation_counts_df.columns = ['Occupation', 'Count']
## there are 250 different occupation in over_threshold
pd.set_option('display.max_rows', None)
print("Occupation Counts as DataFrame:")
display(occupation_counts_df)
display(over_threshold['Occupation'].describe())
display(result_df['Occupation'].describe())

Occupation Counts as DataFrame:


,Occupation,Count
0,0,15383
1,"Freelancer (e.g., Graphic Designer, Writer)",2129
2,Construction Contractor,1253
3,Real Estate Agent,1245
4,Landlord,1213
5,Import/Export Dealer,931
6,Property Manager,894
7,Other,819
8,Auto Dealer,748
9,Banker,738


count     52353
unique      250
top           0
freq      15383
Name: Occupation, dtype: int64

count     300008
unique       251
top            0
freq      104219
Name: Occupation, dtype: int64

### determine transaction on risk people
by observation, some people have same wire_send_aboard,	wire_send_domestic, and same	wire_receive_aboard,wire_receive_domestic, we consider those scenario as risk people

In [35]:
over_threshold.head()

,Name,Gender,Occupation,Age,Tenure,cust_id,label,amount_withdraw,amount_deposite,emt_sender,emt_receive,wire_send_aboard,wire_send_domestic,wire_receive_aboard,wire_receive_domestic,total_amount
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0.0,0.0,0.0,14412.0,11484.5,3971.5,0.0,0.0,0.0,29868.0
3,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,0.0,0.0,3835.0,4494.0,6538.5,28263.0,28263.0,22629.5,22629.5,116652.5
7,STEPHEN FIGUEROA,male,Optometrist,35.0,17.0,CUST33995820,0.0,650.0,0.0,0.0,0.0,0.0,0.0,0.0,27385.0,28035.0
8,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1.0,19585.0,68115.0,1147.5,18289.0,10429.0,0.0,23895.5,0.0,141461.0
16,ZHANG FENG,female,Stock Broker,23.0,5.0,CUST89345714,0.0,4335.0,0.0,250.0,2848.0,2641.0,0.0,0.0,0.0,10074.0


In [37]:
risk_people = over_threshold[(over_threshold['wire_send_aboard'] == \
                                over_threshold['wire_send_domestic']) & \
                                (over_threshold['wire_receive_aboard'] ==
                                over_threshold['wire_receive_domestic']) & \
                                (over_threshold['wire_send_aboard'] > 0) & \
                                (over_threshold['wire_receive_aboard'] > 0) & \
                                (over_threshold['wire_send_domestic'] > 0) & \
                                (over_threshold['wire_receive_domestic'] > 0)]

risk_people.head()

,Name,Gender,Occupation,Age,Tenure,cust_id,label,amount_withdraw,amount_deposite,emt_sender,emt_receive,wire_send_aboard,wire_send_domestic,wire_receive_aboard,wire_receive_domestic,total_amount
3,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,0.0,0.0,3835.0,4494.0,6538.5,28263.0,28263.0,22629.5,22629.5,116652.5
62,FANG JIAN JUN,female,Private Banker,35.0,3.0,CUST14940080,1.0,22045.0,26120.0,8019.0,1141.5,40495.5,40495.5,19047.5,19047.5,176411.5
586,DR.ZHU LIANG,female,Import/Export Business Owner,58.0,0.0,CUST60050014,1.0,0.0,32370.0,1425.0,5450.0,12476.0,12476.0,32123.0,32123.0,128443.0
1570,NATASHA VASQUEZ MD,female,International Salesperson,45.0,18.0,CUST64907257,0.0,33075.0,17730.0,4860.5,3834.0,41679.5,41679.5,13002.0,13002.0,168862.5
1822,THOMAS PRICE,other,Luxury Yacht Broker,41.0,20.0,CUST80080798,0.0,0.0,0.0,746.0,5391.0,12572.0,12572.0,24545.5,24545.5,80372.0
